In [1]:
import numpy as np
import pandas as pd
import json

from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/laxmimerit/twitter-data/master/twitt30k.csv')
df

,twitts,sentiment
0,@robbiebronniman Sounds like a great night.,1
1,Damn the person who stolde my wallet !!!!! Ma...,1
2,Greetings from the piano bench (photo) http:/...,1
3,@drewryanscott i love it!! i love you!! haha f...,1
4,"@kissthestars Pretty pretty pretty please, pak...",0
...,...,...
29995,@Calumfan1 is it in any way related to photosh...,0
29996,@Swiz_NZ really? wow thats crap,0
29997,"At the 2010 lexus HS250h press event. Again, ...",0
29998,@karmicunderpath ooooh now there's a nice thou...,1


In [3]:
df.isna().sum()

twitts       0
sentiment    0
dtype: int64

In [4]:
df.sentiment.value_counts()

0    15000
1    15000
Name: sentiment, dtype: int64

In [5]:
def run_svm(df):
    X=df['twitts']
    y=df['sentiment']
    
    tfidf = TfidfVectorizer()
    X = tfidf.fit_transform(df['twitts'])
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0, stratify = y)
                            
    print('shape of X: ', X.shape)                        
    
    #Train Model
    clf = LinearSVC()
    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)

    print()
    print('Printing Report')
    print(classification_report(y_test, y_pred))
    
    return tfidf, clf
    

In [6]:
%%time
tfidf, clf = run_svm(df)

shape of X:  (30000, 40854)

Printing Report
              precision    recall  f1-score   support

           0       0.75      0.74      0.75      3000
           1       0.74      0.75      0.75      3000

    accuracy                           0.75      6000
   macro avg       0.75      0.75      0.75      6000
weighted avg       0.75      0.75      0.75      6000

CPU times: user 496 ms, sys: 13 ms, total: 510 ms
Wall time: 507 ms


In [7]:
x = ['i am really happy. thanks a lot for coming with me']

In [8]:
clf.predict(tfidf.transform(x))

array([1])

# Data Cleaning and Retraining SVM

In [9]:
import preprocess_andy as pp

In [10]:
dir(pp)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 'get_avg_wordlength',
 'get_characcounts',
 'get_cont_exp',
 'get_digit_counts',
 'get_emails',
 'get_hashtag_counts',
 'get_mentions_counts',
 'get_stopwords_counts',
 'get_uppercase_counts',
 'get_urls',
 'get_value_counts',
 'get_wordcount',
 'make_to_base',
 'remove_accented_chars',
 'remove_common_words',
 'remove_emails',
 'remove_html_tags',
 'remove_rarewords',
 'remove_rt',
 'remove_special_chars',
 'remove_stopwords',
 'remove_urls',
 'spelling_correction',
 'utils']

# Apply lower and Contractions

In [11]:
df['twitts'] = df['twitts'].apply(lambda x: x.lower())

In [12]:
df['twitts'] = df['twitts'].apply(lambda x: pp.get_cont_exp(x))

In [13]:
df.head()

,twitts,sentiment
0,@robbiebronniman sounds like a great night.,1
1,damn the person who stolde my wallet !!!!! ma...,1
2,greetings from the piano bench (photo) http:/...,1
3,@drewryanscott i love it!! i love you!! haha f...,1
4,"@kissthestars pretty pretty pretty please, pak...",0


In [14]:
run_svm(df)

shape of X:  (30000, 40846)

Printing Report
              precision    recall  f1-score   support

           0       0.75      0.74      0.75      3000
           1       0.75      0.76      0.75      3000

    accuracy                           0.75      6000
   macro avg       0.75      0.75      0.75      6000
weighted avg       0.75      0.75      0.75      6000



(TfidfVectorizer(), LinearSVC())

# REMOVE EMAILS

In [15]:
df['twitts'] = df['twitts'].apply(lambda x: pp.remove_emails(x))
df['twitts'] = df['twitts'].apply(lambda x: pp.remove_urls(x))
df['twitts'] = df['twitts'].apply(lambda x: pp.remove_rt(x))
df['twitts'] = df['twitts'].apply(lambda x: pp.remove_html_tags(x))
df['twitts'] = df['twitts'].apply(lambda x: pp.remove_special_chars(x))

In [16]:
df

,twitts,sentiment
0,robbiebronniman sounds like a great night,1
1,damn the person who stolde my wallet may karma...,1
2,greetings from the piano bench photo,1
3,drewryanscott i love it i love you haha forget...,1
4,kissthestars pretty pretty pretty please pakid...,0
...,...,...
29995,calumfan1 is it in any way related to photoshop,0
29996,swiz_nz really wow thats crap,0
29997,at the 2010 lexus hs250h press event again can...,0
29998,karmicunderpath ooooh now there is a nice thought,1


In [17]:
%%time
tfidf, clf = run_svm(df)

shape of X:  (30000, 42983)

Printing Report
              precision    recall  f1-score   support

           0       0.75      0.74      0.74      3000
           1       0.74      0.75      0.75      3000

    accuracy                           0.75      6000
   macro avg       0.75      0.75      0.75      6000
weighted avg       0.75      0.75      0.75      6000

CPU times: user 442 ms, sys: 7.78 ms, total: 450 ms
Wall time: 448 ms


In [18]:
clf.predict(tfidf.transform(x))

array([1])

# FINE TUNING MODEL

In [19]:
def run_svm(df):
    X = df['twitts']
    y = df['sentiment']

    tfidf = TfidfVectorizer(norm = 'l1', ngram_range=(1,2), analyzer='word', max_features=5000)
    X = tfidf.fit_transform(X)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0, stratify = y)

    print('shape of X: ', X.shape)

    clf = LinearSVC()
    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)

    print()
    print('Printing Report')
    print(classification_report(y_test, y_pred))
    
    return tfidf, clf

run_svm(df)

shape of X:  (30000, 5000)

Printing Report
              precision    recall  f1-score   support

           0       0.75      0.77      0.76      3000
           1       0.77      0.75      0.76      3000

    accuracy                           0.76      6000
   macro avg       0.76      0.76      0.76      6000
weighted avg       0.76      0.76      0.76      6000



(TfidfVectorizer(max_features=5000, ngram_range=(1, 2), norm='l1'),
 LinearSVC())

# SAVING AND LOADING THE MODEL

In [20]:
import pickle

In [21]:
pickle.dump(clf, open('clf.pkl', 'wb'))
pickle.dump(tfidf, open('tfidf.pkl', 'wb'))

In [22]:
del clf
del tfidf

In [23]:
clf = pickle.load(open('clf.pkl', 'rb'))
tfidf = pickle.load(open('tfidf.pkl', 'rb'))

In [24]:
#tfidf.vocabulary_

In [25]:
clf.predict(tfidf.transform(x))

array([1])

# REAL TIME SENTIMENT ANALYSIS

In [26]:
consumer_key = 'i47xjSBBe07R8zGauaQbl7oBA'
consumer_secret = 'sZoeCAoTO5zrHi270dBhhQhL203I6kQUjfKSRABrvQTxeSH9Zj'

In [27]:
access_token = '232661448-abXPKetKXDIWIU7RHZwstkNxw7vlONjKgjhDBEM7'
access_token_secret = 'mxJbEvny2kv3UsP6DkwSQAG11VlYnZJy3O2s0ywdYOl53'

In [28]:
import tweepy

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

public_tweets = api.home_timeline()
for tweet in public_tweets:
    print(tweet.text)

$DIA $SPX $SPY $COMPQ: Equity Markets - 05/14/21 The inflation-fears driven drop in global stocks is taking a breat… https://t.co/X1g2FUeokK
Klarna turns to murals and #QR codes to “set the record straight” https://t.co/mgJRiAvXaV

@altfinews via… https://t.co/1ro17pQdvQ
RT @vanivina9: @star95011 @dansar39 @bryankramer @Nicochan33 @DrJDrooghaag @AkwyZ @SylviaLobo @IanLJones98 @KirkDBorne @terence_mills @Dami…
RT @Ronald_vanLoon: 13 years ago today, #SteveJobs unveiled the original #iPhone  changing millions of lives and devices across the world.…
@enricomolinari @cgledhill @leimer @FrancisPouliot @mi_jim @WSWMUC @KirkDBorne @KimWhitler @KenLittle… https://t.co/6UDyfVzOyl
Ant leapfrogs banks to top China fund sale rankings https://t.co/2x9I0EmeJq https://t.co/kofbe7A2al
The U.S. takes its vaccination drive to the factory floor https://t.co/oBmg8idAq0
RT @enricomolinari: #FF #training on the top!! #edtech 

@cgledhill @leimer @francispouliot @mi_jim @WSWMUC @KirkDBorne @KimWhitler @KenLi

# TRACKING KEYWORDS ON TWITTER

In [29]:
import json
import pickle
from textblob import TextBlob
import csv

In [30]:
import preprocess_andy as pp

In [31]:
clf = pickle.load(open('clf.pkl', 'rb'))
tfidf = pickle.load(open('tfidf.pkl', 'rb'))

In [32]:
def predict_sentiment(x):
    x = [x]
    sent = clf.predict(tfidf.transform(x))
    return sent

In [33]:
predict_sentiment('how are you')[0]

1

In [34]:
track_keyword = ['usa','china']

In [35]:
usa = 0
china = 0

with open('senti.csv','w') as file:
    writer = csv.DictWriter(file,fieldnames=track_keyword)
    writer.writeheader()

class MyStreamListener(tweepy.StreamListener):
    def on_status(self, status):
        print(status.text)

    def on_data(self, data):
        raw_twitt = json.loads(data)
        try:
            x = str(raw_twitt['text']).lower()
            x = pp.get_cont_exp(x)
            x = pp.remove_emails(x)
            x = pp.remove_html_tags(x)
            x = pp.remove_rt(x)
            x= pp.remove_special_chars(x)
            x = pp.remove_urls(x)
            
            
            #blob = TextBlob(x)
            sentiment = predict_sentiment(x)[0]
            print(sentiment)
            global china
            global usa
            
            if 'usa' in x and 'china' not in x:
                sent = predict_sentiment(x)[0]
                usa = usa + sent
                
            elif 'china' in x and 'usa' not in x:
                sent = predict_sentiment(x)[0]
                china = china + sent
        except:
            pass
        print('usa: ', usa, 'china: ', china)
        
        with open('senti.csv', 'a') as file:
                writer = csv.DictWriter(file, fieldnames=track_keyword)
                info = {
                    'usa': usa,
                    'china': china
                }
                writer.writerow(info)

    def on_error(self, status_code):
        if status_code == 420:
            #returning False in on_error disconnects the stream
            return False

In [36]:
myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth = api.auth, listener=myStreamListener)

In [37]:
myStream.filter(track=track_keyword)

1
usa:  0 china:  1
0
usa:  0 china:  1
1
usa:  0 china:  1
1
usa:  1 china:  1
1
usa:  1 china:  1
0
usa:  1 china:  1
1
usa:  2 china:  1
0
usa:  2 china:  1
1
usa:  2 china:  1
0
usa:  2 china:  1
0
usa:  2 china:  1
1
usa:  2 china:  1
1
usa:  2 china:  1
1
usa:  3 china:  1
0
usa:  3 china:  1
0
usa:  3 china:  1
1
usa:  3 china:  1
1
usa:  3 china:  2
1
usa:  3 china:  2
0
usa:  3 china:  2
1
usa:  3 china:  2
1
usa:  3 china:  2
1
usa:  3 china:  2
0
usa:  3 china:  2
1
usa:  3 china:  2
1
usa:  3 china:  2
0
usa:  3 china:  2


/usr/local/anaconda3/envs/nlp_course/lib/python3.7/site-packages/bs4/__init__.py:421: MarkupResemblesLocatorWarning: "https://t.co/jscvf9ko6x" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  MarkupResemblesLocatorWarning


1
usa:  4 china:  2
1
usa:  4 china:  2
1
usa:  4 china:  2
1
usa:  5 china:  2
1
usa:  5 china:  3
1
usa:  5 china:  3
0
usa:  5 china:  3
0
usa:  5 china:  3
1
usa:  5 china:  4
1
usa:  5 china:  4
0
usa:  5 china:  4
0
usa:  5 china:  4
0
usa:  5 china:  4
1
usa:  6 china:  4
0
usa:  6 china:  4
1
usa:  6 china:  5
0
usa:  6 china:  5
1
usa:  6 china:  6
0
usa:  6 china:  6
0
usa:  6 china:  6
0
usa:  6 china:  6
1
usa:  7 china:  6
1
usa:  8 china:  6
0
usa:  8 china:  6
1
usa:  8 china:  6
0
usa:  8 china:  6
1
usa:  8 china:  6
1
usa:  8 china:  6
1
usa:  8 china:  6
0
usa:  8 china:  6
0
usa:  8 china:  6
0
usa:  8 china:  6
1
usa:  8 china:  6
1
usa:  9 china:  6
1
usa:  9 china:  7
0
usa:  9 china:  7
1
usa:  10 china:  7
1
usa:  11 china:  7
0
usa:  11 china:  7
1
usa:  12 china:  7
1
usa:  12 china:  7
0
usa:  12 china:  7
1
usa:  13 china:  7
0
usa:  13 china:  7
1
usa:  13 china:  8
0
usa:  13 china:  8
1
usa:  14 china:  8
1
usa:  14 china:  8
0
usa:  14 china:  8
1
usa: 

1
usa:  86 china:  47
1
usa:  86 china:  48
1
usa:  87 china:  48
0
usa:  87 china:  48
1
usa:  88 china:  48
0
usa:  88 china:  48
1
usa:  88 china:  48
1
usa:  88 china:  48
0
usa:  88 china:  48
0
usa:  88 china:  48
1
usa:  88 china:  49
0
usa:  88 china:  49
1
usa:  88 china:  49
0
usa:  88 china:  49
0
usa:  88 china:  49
0
usa:  88 china:  49
1
usa:  89 china:  49
1
usa:  89 china:  50
0
usa:  89 china:  50
0
usa:  89 china:  50
0
usa:  89 china:  50
0
usa:  89 china:  50
1
usa:  89 china:  50
1
usa:  89 china:  51
0
usa:  89 china:  51
0
usa:  89 china:  51
1
usa:  89 china:  51
1
usa:  90 china:  51
1
usa:  90 china:  51
1
usa:  90 china:  51
1
usa:  90 china:  51
0
usa:  90 china:  51
0
usa:  90 china:  51
0
usa:  90 china:  51
0
usa:  90 china:  51
1
usa:  90 china:  51
0
usa:  90 china:  51
1
usa:  90 china:  52
1
usa:  91 china:  52
1
usa:  91 china:  52
0
usa:  91 china:  52
1
usa:  92 china:  52
0
usa:  92 china:  52
1
usa:  92 china:  53
1
usa:  92 china:  53
1
usa:  92

1
usa:  156 china:  85
1
usa:  156 china:  85
0
usa:  156 china:  85
1
usa:  156 china:  85
0
usa:  156 china:  85
0
usa:  156 china:  85
1
usa:  157 china:  85
1
usa:  157 china:  86
1
usa:  157 china:  86
1
usa:  158 china:  86
1
usa:  158 china:  86
0
usa:  158 china:  86
0
usa:  158 china:  86
1
usa:  159 china:  86
1
usa:  160 china:  86
1
usa:  161 china:  86
1
usa:  161 china:  86
1
usa:  161 china:  86
0
usa:  161 china:  86
1
usa:  161 china:  87
0
usa:  161 china:  87
1
usa:  161 china:  87
1
usa:  161 china:  87
0
usa:  161 china:  87
1
usa:  161 china:  87
1
usa:  162 china:  87
1
usa:  162 china:  88
0
usa:  162 china:  88
1
usa:  162 china:  88
1
usa:  163 china:  88
0
usa:  163 china:  88
1
usa:  163 china:  88
1
usa:  163 china:  88
0
usa:  163 china:  88
1
usa:  163 china:  89
1
usa:  163 china:  89
1
usa:  163 china:  89
1
usa:  164 china:  89
0
usa:  164 china:  89
0
usa:  164 china:  89
0
usa:  164 china:  89
1
usa:  164 china:  89
1
usa:  165 china:  89
1
usa:  166

...

/usr/local/anaconda3/envs/nlp_course/lib/python3.7/site-packages/bs4/__init__.py:421: MarkupResemblesLocatorWarning: "https://t.co/ao1zsbwkfu" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  MarkupResemblesLocatorWarning


0
usa:  955 china:  527
0
usa:  955 china:  527
0
usa:  955 china:  527
1
usa:  955 china:  527
1
usa:  956 china:  527
0
usa:  956 china:  527
1
usa:  957 china:  527
1
usa:  958 china:  527
1
usa:  958 china:  527
1
usa:  959 china:  527
1
usa:  959 china:  527
0
usa:  959 china:  527
0
usa:  959 china:  527
1
usa:  959 china:  527
1
usa:  959 china:  527
1
usa:  959 china:  527
0
usa:  959 china:  527
1
usa:  960 china:  527
0
usa:  960 china:  527
0
usa:  960 china:  527
1
usa:  960 china:  527
1
usa:  960 china:  528
0
usa:  960 china:  528
0
usa:  960 china:  528
1
usa:  960 china:  528
0
usa:  960 china:  528
0
usa:  960 china:  528
0
usa:  960 china:  528
0
usa:  960 china:  528
0
usa:  960 china:  528
0
usa:  960 china:  528
0
usa:  960 china:  528
1
usa:  961 china:  528
1
usa:  961 china:  528
0
usa:  961 china:  528
1
usa:  961 china:  528
0
usa:  961 china:  528
0
usa:  961 china:  528
0
usa:  961 china:  528
1
usa:  961 china:  528
0
usa:  961 china:  528
1
usa:  961 chin

/usr/local/anaconda3/envs/nlp_course/lib/python3.7/site-packages/bs4/__init__.py:421: MarkupResemblesLocatorWarning: "https://t.co/kulquypmap" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  MarkupResemblesLocatorWarning


1
usa:  984 china:  537
1
usa:  985 china:  537
0
usa:  985 china:  537
0
usa:  985 china:  537
0
usa:  985 china:  537
1
usa:  986 china:  537
1
usa:  986 china:  537
1
usa:  986 china:  537
0
usa:  986 china:  537
1
usa:  986 china:  537
0
usa:  986 china:  537
1
usa:  986 china:  537
0
usa:  986 china:  537
0
usa:  986 china:  537
0
usa:  986 china:  537
1
usa:  987 china:  537
0
usa:  987 china:  537
0
usa:  987 china:  537
1
usa:  988 china:  537
0
usa:  988 china:  537
1
usa:  988 china:  538
0
usa:  988 china:  538
0
usa:  988 china:  538
0
usa:  988 china:  538
1
usa:  989 china:  538
1
usa:  989 china:  538
0
usa:  989 china:  538
1
usa:  990 china:  538
1
usa:  991 china:  538
1
usa:  991 china:  538
1
usa:  992 china:  538
1
usa:  993 china:  538
1
usa:  994 china:  538
0
usa:  994 china:  538
1
usa:  994 china:  538
1
usa:  995 china:  538
1
usa:  995 china:  539
1
usa:  995 china:  539
1
usa:  996 china:  539
0
usa:  996 china:  539
1
usa:  996 china:  540
1
usa:  996 chin

ProtocolError: ('Connection broken: IncompleteRead(0 bytes read)', IncompleteRead(0 bytes read))